Importamos librerias

In [3]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../../')
from utils_yose import build_preprocessor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import mlflow

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.5f' % x)